In [1]:
import extract_data
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import h5py
from scipy.signal import filtfilt, butter, iirnotch, welch
import math
from collections import deque
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix

In [2]:
data, words, starts, ends = extract_data.extract_data(r"C:\Users\lwing\Downloads\College\Spring 2022\Senior Design II\SilentSpeechDAS\fEMGData\mouthed_full_6_set1.txt", r"C:\Users\lwing\Downloads\College\Spring 2022\Senior Design II\silent-speech\scripts\Mouthed_Full_6_set1.txt")


2022-04-01T15:59:48.241-05


In [3]:
chan_1 = data[:,0]
chan_2 = data[:,1]
chan_3 = data[:,2]
chan_4 = data[:,3]
chan_5 = data[:,4]
chan_6 = data[:,5]
channel_data = [chan_1, chan_2, chan_3, chan_4, chan_5, chan_6]

### Creating target variable list

In [4]:
len(chan_1)

791100

In [5]:
y = np.zeros(791100)

In [6]:
for i in range(len(starts)):
    y[starts[i]:ends[i]+1] = 1

In [7]:
print(np.count_nonzero(y))
print(len(y))

186972
791100


### Filtering

In [8]:
# [Band Pass to demonstrate most prominent frequency range]
low_cutoff = 20
high_cutoff = 450

filtered_data = [0,0,0,0,0,0]

for idx, channel in enumerate(channel_data):
    signal_meancorrect = channel - np.mean(channel)
    
    #60Hz Notch Filter for Power Line Noise
    b, a = iirnotch(60, 30, 1000)
    signal_notched = filtfilt(b, a, signal_meancorrect)

    # Fourth Order Butterworth 
    b, a = butter(10, [low_cutoff, high_cutoff], fs=1000, btype='bandpass')
    signal_filtered = filtfilt(b, a, signal_notched)

    #Rectify signal
    filtered_data[idx] = abs(signal_filtered)

### Deleting Extraneous Data

In [9]:
#deleting indices :24887,486094-504933, 763442:

for i in range(len(filtered_data)):
    arr1 = filtered_data[i][24887:486094]
    arr2 = filtered_data[i][504933:763442]
    filtered_data[i] = np.concatenate([arr1, arr2])
    

In [10]:
arr1 = y[24887:486094]
arr2 = y[504933:763442]
y = np.concatenate([arr1, arr2])


### Downsampling y

In [11]:
#large window to average over
window = 40
        
        #overlap interval
skip = 20
        
ind1 = 0
ind2 = window
y_new = np.zeros(int(len(y)/20)+1)
i = 0
while ind1 < len(y):
            
    #remaining data less than window size, avoid array out of bounds
    if ind2 > len(y):
        ind2 = len(y)-1
                
    num_label = np.count_nonzero(y[ind1:ind2])
    if num_label > 19:
        y_new[i] = 1
    ind1 = ind1 + skip
    ind2 = ind2 + skip
    i = i+1

In [12]:
len(y_new)

35986

### Prepping filtered data with rms, smoothing, and downsampling operations

In [22]:
def rms(raw):
        rms_window = deque([0,0,0,0,0])
        rms_data = np.zeros(len(raw))
        for i, sample in enumerate(raw):
            rms_window.popleft()
            rms_window.append(sample)
            val = np.sqrt(sum(np.square(rms_window)/5))
            rms_data[i] = val
            
        return rms_data
    
    
#TO DO: MAKE SURE ARRAY OUT OF BOUNDS CHECK IS SUFFICIENT
def smooth(rms_data):
        
    #large window to average over; sampling rate is 1000 Hz; each sample is a millisecond
    window = 40
        
    #overlap interval
    skip = 20
        
    ind1 = 0
    ind2 = window
    #assuming that the packet size i.e. length of raw data and rms_data will be a multiple of 20
    downsampled = np.zeros(int(len(rms_data)/20)+1)
    i = 0
    while ind1 < len(rms_data):
            
        #remaining data less than window size, avoid array out of bounds
        if ind2 > len(rms_data):
            ind2 = len(rms_data)-1
                
        val = np.mean(rms_data[ind1:ind2], dtype=np.float64)
        downsampled[i] = val
        ind1 = ind1 + skip
        ind2 = ind2 + skip
        i = i+1
            
    return downsampled
        
def calculate(smoothed_envelope):
        
    return np.abs(np.diff(smoothed_envelope))

In [26]:
ready_data = [0,0,0,0,0,0]

for i in range(len(filtered_data)):
    data_rms = rms(filtered_data[i])
    res = smooth(data_rms)
    ready_data[i] = calculate(res)

In [27]:
print(np.count_nonzero(ready_data[0]))
print(len(ready_data[0]))

35985
35985


In [28]:
y_new = y_new[:35985]

### FSM Testing Thresholds

In [183]:
#.025, 5
#meme result .01, 1.5
channel_results = np.zeros((6,35985))

for idx, channel in enumerate(ready_data):
    active = False
    max_power = 0
    min_power = 1
    max_thresh = .01
    min_thresh = 1.5
    for index, sample in enumerate(channel):
        
        if active:
            if sample > (max_power * max_thresh):
                active = True
                channel_results[idx,index] = 1
            else:
                active = False
                channel_results[idx,index] = 0
            if sample > max_power:
                max_power = sample
            
            
        else:
            if sample < (min_power * min_thresh):
                active = False
                channel_results[idx,index] = 0
            else:
                active = True
                channel_results[idx,index] = 1
            if sample < min_power:
                min_power = sample

In [216]:
isSpeech = False
inactive_count=0
active_count=0
inactive_thresh = 1
active_thresh = 10

speech_event = np.zeros(35985)

for sample_idx in range(35985):       
    
    num_active = np.count_nonzero(channel_results[:,sample_idx] == 1)
    
            
    if isSpeech:
        if num_active < 4:
            inactive_count += 1
        else:
            inactive_count = 0
            
        if inactive_count > inactive_thresh:
            isSpeech = False
            speech_event[sample_idx] = 0
                    
        else:
            isSpeech = True
            speech_event[sample_idx] = 1
     
        
    else:
        #updating speech event count
        if num_active >= 4:
            active_count += 1
        else:
            active_count = 0
        
        #Speech Event conditional
        if active_count > active_thresh:
            isSpeech = True
            speech_event[sample_idx] = 1
                        
        else:
            isSpeech = False
            speech_event[sample_idx] = 0

In [217]:

accuracy = accuracy_score(y_new,speech_event)
cm=confusion_matrix(y_new, speech_event) 
print(f"Accuracy = {accuracy:.5f}, Confusion Matrix = {cm}")

Accuracy = 0.44174, Confusion Matrix = [[ 8017 18841]
 [ 1248  7879]]


### Testing on unseen data

In [218]:
data_test, words_test, starts_test, ends_test = extract_data.extract_data(r"C:\Users\lwing\Downloads\College\Spring 2022\Senior Design II\SilentSpeechDAS\fEMGData\mouthed_full_6_set2.txt", r"C:\Users\lwing\Downloads\College\Spring 2022\Senior Design II\silent-speech\scripts\Mouthed_Full_6_set2.txt")


2022-04-01T16:17:14.794-05


In [219]:
chan_1_test = data_test[:,0]
chan_2_test = data_test[:,1]
chan_3_test = data_test[:,2]
chan_4_test = data_test[:,3]
chan_5_test = data_test[:,4]
chan_6_test = data_test[:,5]
channel_data_test = [chan_1_test, chan_2_test, chan_3_test, chan_4_test, chan_5_test, chan_6_test]

In [220]:
len(chan_1_test)

762300

In [221]:
y_test = np.zeros(762300)

In [222]:
for i in range(len(starts_test)):
    y_test[starts_test[i]:ends_test[i]+1] = 1

In [223]:
# [Band Pass to demonstrate most prominent frequency range]
low_cutoff = 20
high_cutoff = 450

filtered_data_test = [0,0,0,0,0,0]

for idx, channel in enumerate(channel_data_test):
    signal_meancorrect = channel - np.mean(channel)
    
    #60Hz Notch Filter for Power Line Noise
    b, a = iirnotch(60, 30, 1000)
    signal_notched = filtfilt(b, a, signal_meancorrect)

    # Fourth Order Butterworth 
    b, a = butter(10, [low_cutoff, high_cutoff], fs=1000, btype='bandpass')
    signal_filtered = filtfilt(b, a, signal_notched)

    #Rectify signal
    filtered_data_test[idx] = abs(signal_filtered)

In [224]:
for i in range(len(filtered_data_test)):
    filtered_data_test[i] = filtered_data_test[i][20750:747706]

In [225]:
len(filtered_data_test[0])

726956

In [226]:
y_test = y_test[20750:747706]

In [227]:
#large window to average over
window = 40
        
        #overlap interval
skip = 20
        
ind1 = 0
ind2 = window
y_test_new = np.zeros(int(len(y_test)/20)+1)
i = 0
while ind1 < len(y_test):
            
    #remaining data less than window size, avoid array out of bounds
    if ind2 > len(y_test):
        ind2 = len(y_test)-1
                
    num_label = np.count_nonzero(y_test[ind1:ind2])
    if num_label > 19:
        y_test_new[i] = 1
    ind1 = ind1 + skip
    ind2 = ind2 + skip
    i = i+1

In [229]:
ready_data_test = [0,0,0,0,0,0]

for i in range(len(filtered_data_test)):
    data_rms = rms(filtered_data_test[i])
    res = smooth(data_rms)
    ready_data_test[i] = calculate(res)

In [230]:
print(np.count_nonzero(ready_data_test[0]))
print(len(ready_data_test[0]))

36347
36347


In [238]:
y_test_new = y_test_new[:36347]

In [239]:
channel_results = np.zeros((6,36347))

for idx, channel in enumerate(ready_data_test):
    active = False
    max_power = 0
    min_power = 1
    max_thresh = .01
    min_thresh = 1.5
    for index, sample in enumerate(channel):
        
        if active:
            if sample > (max_power * max_thresh):
                active = True
                channel_results[idx,index] = 1
            else:
                active = False
                channel_results[idx,index] = 0
            if sample > max_power:
                max_power = sample
            
            
        else:
            if sample < (min_power * min_thresh):
                active = False
                channel_results[idx,index] = 0
            else:
                active = True
                channel_results[idx,index] = 1
            if sample < min_power:
                min_power = sample

In [240]:
isSpeech = False
inactive_count=0
active_count=0
inactive_thresh = 1
active_thresh = 10

speech_event = np.zeros(36347)

for sample_idx in range(36347):       
    
    num_active = np.count_nonzero(channel_results[:,sample_idx] == 1)
    
            
    if isSpeech:
        if num_active < 4:
            inactive_count += 1
        else:
            inactive_count = 0
            
        if inactive_count > inactive_thresh:
            isSpeech = False
            speech_event[sample_idx] = 0
                    
        else:
            isSpeech = True
            speech_event[sample_idx] = 1
     
        
    else:
        #updating speech event count
        if num_active >= 4:
            active_count += 1
        else:
            active_count = 0
        
        #Speech Event conditional
        if active_count > active_thresh:
            isSpeech = True
            speech_event[sample_idx] = 1
                        
        else:
            isSpeech = False
            speech_event[sample_idx] = 0

In [243]:
accuracy = accuracy_score(y_test_new,speech_event)
cm=confusion_matrix(y_test_new, speech_event) 
print(f"Accuracy = {accuracy:.5f}, Confusion Matrix = {cm}")

Accuracy = 0.42788, Confusion Matrix = [[ 8079 19522]
 [ 1273  7473]]
